Pu pratices his Pytorch skills by writing a classification algorithm using Pytorch. He also uses sklearn's LabelEncoder to transform categorical features. This is based on the tutorial on this page: https://machinelearningmastery.com/use-pytorch-deep-learning-models-with-scikit-learn/

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

# read the data
data = pd.read_csv("../data/label-encoder/sonar.csv")
X = data.iloc[:, :60]
y = data.iloc[:, 60]

encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)


X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [17]:
# define the model
import torch.nn as nn

class SonarClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(60, 60)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(60, 60)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(60, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.output(x)
        return x

In [18]:
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

batch_size = 5
learning_rate = 0.01
num_epochs = 50

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, shuffle=True)
train, test = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

model = SonarClassifier()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCEWithLogitsLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device).view(-1, 1)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'Epoch: [{epoch} / {num_epochs}], loss: {loss.item()}')
    with torch.no_grad():
        num_samples, num_correct = 0, 0
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device).view(-1, 1)
            outputs = model(inputs)
            predicted = torch.round(torch.sigmoid(outputs))
            num_correct += (predicted == targets).float().sum().item()
            num_samples += targets.size(0)
        acc = num_correct / num_samples
        print(f"Accuracy of the network on test data: {acc * 100} %")


Epoch: [0 / 50], loss: 0.6922688484191895
Epoch: [0 / 50], loss: 0.6807352304458618
Accuracy of the network on test data: 51.369863013698634 %
Epoch: [1 / 50], loss: 0.6728610396385193
Epoch: [1 / 50], loss: 0.5479284524917603
Accuracy of the network on test data: 51.369863013698634 %
Epoch: [2 / 50], loss: 0.6523057818412781
Epoch: [2 / 50], loss: 0.4977099597454071
Accuracy of the network on test data: 51.369863013698634 %
Epoch: [3 / 50], loss: 0.6471546292304993
Epoch: [3 / 50], loss: 0.2094271183013916
Accuracy of the network on test data: 67.8082191780822 %
Epoch: [4 / 50], loss: 0.1978704184293747
Epoch: [4 / 50], loss: 0.37108534574508667
Accuracy of the network on test data: 56.16438356164384 %
Epoch: [5 / 50], loss: 0.20211659371852875
Epoch: [5 / 50], loss: 0.6669803261756897
Accuracy of the network on test data: 76.71232876712328 %
Epoch: [6 / 50], loss: 0.4944819509983063
Epoch: [6 / 50], loss: 0.471902996301651
Accuracy of the network on test data: 73.97260273972603 %
Epo